# Unit 3 服务 Part 1 解决方案

<img src="../img/robotignite_logo_text.png"/>

## 索引: 
* <a href="#SolutionExercise3-1"练习 3.1解决方案</a>

## 练习3.1 解决方案<p id="SolutionExercise3-1"></p>

* 首先，创建一个功能包存放将来所有的代码。为了将来参考方便，可以命名为 **unit_3_services**, 依赖项为 **rospy** 和**iri_wam_reproduce_trajectory** 。

首先创建一个新的功能包，该功能包用来存放所有与**Services** 主题相关的代码。Unit3的Part1和Part2都会用到该功能包。

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

* 创建一个名为**my_robot_arm_demo.launch** 的launch文件来开启 **/execute_strajectory** 服务。如在 <a href="#example3.3">example 3.3</a>中解释的那样，该服务由功能包**iri_wam_reproduce_trajectory** 中的**start_service.launch** 启动。

接着，我们创建一个launch文件，命名为**my_robot_arm_demo.launch**.<br>
在该文件中，我们将添加所有完成该练习所需要的启动项。

<p style="background:#3B8F10;color:white;" id="prg-3-1">**Launch File: my_robot_arm_demo.launch** </p>

In [ ]:
<launch>

  <include file="$(find iri_wam_reproduce_trajectory)/launch/start_service.launch"/>

  <!-- Here will go our python script that calls the execute_trajectory service -->

</launch>

<p style="background:#3B8F10;color:white;" id="prg-3-1">END **Launch File: my_robot_arm_demo.launch** </p>

完成后，启动它，检查服务 **execute_trajectory** 是否已经工作：

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch unit_3_services my_robot_arm_demo.launch

确保已经终止了第一个demo的运行，否则系统会在你启动第一个demo的终端提示如下信息：

这是由于，当你启动一个同名的新节点时，有时它会结束前面的那个节点。好的做法是手动结束而不是依赖系统来结束旧的节点。

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

你将得到下面的输出：

In [ ]:
user:~/catkin_ws$ rosservice list | grep execute_trajectory
/execute_trajectory

* 获取**execute_trajectory** 服务使用的服务消息类型的信息。 

要做到这一点，你需要运行获取更多服务信息的命令。记住，launch文件**my_robot_arm_demo.launch** 必须确保在运行：

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

你将得到：

这告诉我们，如果我们要调用服务 **execute_trajectory**，我们将需要使用功能包**iri_wam_reproduce_trajectory** 中的服务消息类型**ExecTraj** 。

因此，现在我们需要知道当我们调用该服务时，如何设置**ServiceRequest** 变量。

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

输出应该为：

如你所见，服务消息**ExecTraj** 有：

**Request**: 只有一个名为 **file** 类型为 **String**的变量。字符串（String）是一个基本的消息类型，可以在**std_msgs** 功能包中找到。

**Response**: 本例中该值为空。因此，该服务被调用时，没有响应。这种情况称为**Empty** 影响。

现在我们已经知道了关于该服务的所有信息，接下来就可以开始创建调用它的python程序了。

* 使机械臂按照文件中指定的轨迹运动。<br>
修改名为 **/gazebo/delete_model**的例子 <a href="#example3.5">example 3.5</a> 中的代码，调用 **/execute_trajectory** 服务。新的python文件可以命名为**exercise_3_1.py**。

要修改 **example 3.5**, 你必须更改服务消息的**type** 和**service name**。you have to change the **type** the service message and also the **service name**. 你还需要修改作为请求的变量。也即，修改你要运行的描述机械臂运动轨迹的文件的路径。

<p style="background:#3B8F10;color:white;" id="prg-3-1">**Python File: exercise_3_1.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from iri_wam_reproduce_trajectory.srv import ExecTraj, ExecTrajRequest # Import the service message used by the service /gazebo/delete_model

rospy.init_node('service_execute_trajectory_client') # Initialise a ROS node with the name service_client
rospy.wait_for_service('/execute_trajectory') # Wait for the service client /execute_trajectory to be running
execute_trajectory_service_client = rospy.ServiceProxy('/execute_trajectory', ExecTraj) # Create the connection to the service
execute_trajectory_request_object = ExecTrajRequest() # Create an object of type ExecTrajRequest

"""
user:~/catkin_ws$ rossrv show iri_wam_reproduce_trajectory/ExecTraj
string file
---

"""

execute_trajectory_request_object.file = "file_path" # Fill the variable file of this object with the desired file path
result = execute_trajectory_service_client(execute_trajectory_request_object) # Send through the connection the path to the trajectory file to be executed
print result # Print the result type ExecTrajResponse

<p style="background:#3B8F10;color:white;" id="prg-3-1">END **Python File: exercise_3_1.py** </p>

如你所示，你需要指定**Request** 的变量**file** 的**file_path**。 但是， 这些**trajectory files** 究竟在哪？

* 这里，你有必要根据它们所在的功能包获取trajectory_files文件的路径。本例中选择的trajectory_file是 **get_food.txt**。当然，你可以使用功能包**iri_wam_reproduce_trajectory**里**config**文件夹中任何可用的trajectory_file。

In [ ]:
import rospkg<br>
rospack = rospkg.RosPack()<br>
# This rospack.get_path() works in the same way as $(find name_of_package) in the launch files.<br>
traj = rospack.get_path('iri_wam_reproduce_trajectory') + "/config/get_food.txt"

如上所示，trajecory_files位于功能包 **iri_wam_reproduce_trajectory**中。让我们看一下该功能包，找出轨迹文件所在的文件夹**config**。

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

你将得到：

如果我们使用Web控制台编辑器**vim**查看一下这些**trajectory_files** 中的一个，或者将这些文件中的一个拷贝至我们的工作空间 **/home/user/carkin_ws/src**，你将会明白它们是如何工作的。
让我们查看一下**get_food.txt**文件。

In [ ]:
roscd unit_3_services
mkdir config
roscd iri_wam_reproduce_trajectory/config
cp get_food.txt /home/user/catkin_ws/src/unit_3_services/config

In [ ]:
0.0112138,0.942628,0.133408,1.65916,-0.214736,-1.05983,-0.430974,0.00213521,0.313696,0.0892853,-0.29252,-0.110414,0.0677202,0.140693,-0.00383239,-0.0665293,0.0935825,-0.27529,-0.393899,0.351521,-0.0444471,0.579301,2.77829,-2.87128,0.0812354,-0.336139,-0.000311573,0.0183247,-1.86726,1.48766,3.67008,-1.55314,0.302464,0.355123,0.0429483,0.0256781,-0.100288,0.0139974,-0.00673008,-0.00101568,-0.000844474,-1.56452e-05,-0.687394,-2.01554,2.78754,0.84521,-0.523724,0.667854,0.0283819
0.0112557,0.948886,0.135215,1.65325,-0.217031,-1.0584,-0.428172,0.0020552,0.31212,0.0911877,-0.297971,-0.118286,0.0750198,0.139555,-0.00413107,-0.0884723,0.0962923,-0.269863,-0.392985,0.375216,-0.0669714,0.566718,2.75946,-2.81956,0.0580163,-0.300554,-0.00282943,-0.0253767,-1.86951,1.50267,3.68917,-1.56147,0.306648,0.359253,0.0428217,0.0280618,-0.133746,0.0155503,-0.010118,-0.00110025,-0.0010578,-1.80724e-05,-0.550031,-3.1407,2.94782,-2.77195,-1.55137,0.36242,-0.0223026
0.0112958,0.955107,0.13706,1.64723,-0.219483,-1.05681,-0.425398,0.00196969,0.31013,0.0931383,-0.303284,-0.126112,0.0827353,0.137977,-0.00438779,-0.108162,0.0984662,-0.261782,-0.38954,0.393589,-0.088439,0.591821,2.68984,-2.77308,0.0552174,-0.250296,-0.0489743,0.0163657,-1.8719,1.51752,3.70853,-1.56945,0.311127,0.363222,0.042624,0.0266758,-0.161783,0.0161316,-0.0108963,-0.00112793,-0.00122445,-2.09172e-05,-1.45157,-3.65336,2.54961,-3.98732,-0.158789,0.0574192,0.0163606
0.0113343,0.961284,0.138945,1.64111,-0.22209,-.....

服务**execute_trajectory** 将基于调用中给出的路径读取上面这些值。

因此，我们只需将该文件添加到我们的主python代码exercise_3_1.py：

<p style="background:#3B8F10;color:white;" id="prg-3-1">**Python File: Update exercise_3_1.py** </p>

In [ ]:
#! /usr/bin/env python
import rospkg
import rospy
from iri_wam_reproduce_trajectory.srv import ExecTraj, ExecTrajRequest # Import the service message used by the service /gazebo/delete_model

rospy.init_node('service_execute_trajectory_client') # Initialise a ROS node with the name service_client
rospy.wait_for_service('/execute_trajectory') # Wait for the service client /execute_trajectory to be running
execute_trajectory_service_client = rospy.ServiceProxy('/execute_trajectory', ExecTraj) # Create the connection to the service
execute_trajectory_request_object = ExecTrajRequest() # Create an object of type ExecTrajRequest

"""
user:~/catkin_ws$ rossrv show iri_wam_reproduce_trajectory/ExecTraj
string file
---

"""

rospack = rospkg.RosPack()
# This rospack.get_path() works in the same way as $(find name_of_package) in the launch files.<br>
trajectory_file_path = rospack.get_path('iri_wam_reproduce_trajectory') + "/config/get_food.txt"


execute_trajectory_request_object.file = trajectory_file_path # Fill the variable file of this object with the desired file path
result = execute_trajectory_service_client(execute_trajectory_request_object) # Send through the connection the path to the trajectory file to be executed
print result # Print the result type ExecTrajResponse

<p style="background:#3B8F10;color:white;" id="prg-3-1">**Python File: Update exercise_3_1.py** </p>

* 将**exercise_3_1.py** 的调用添加至主launch文件 **my_robot_arm_demo.launch**。

In [ ]:
<launch>

  <include file="$(find iri_wam_reproduce_trajectory)/launch/start_service.launch"/>

  <!-- Here will go our python script that calls the execute_trajectory service -->
    <node pkg ="unit_3_services"
        type="exercise_3_1.py"
        name="service_execute_trajectory_client"
        output="screen">
  </node>
  
  
</launch>

* 现在，启动 **my_robot_arm_demo.launch** ，看一下机械臂是如何按照指定轨迹运动的。

如果执行的是**get_food.txt**轨迹文件，你将看到下面的运动：

<img src="../img/basic_unit3_exercise3-1.gif"/>